In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import matplotlib.ticker as mticker
import torch

# import svgutils.transform as sg
# from svgutils.compose import *

from plotting_utils import get_size_tuple

from fnope.utils.misc import get_output_dir
import matplotlib as mpl
from pathlib import Path


In [ ]:
#### Unpickle and load data ####

output_dir = get_output_dir()
experiment_folder = output_dir / "darcy_experiment"

# Load data for FNOPE (fix)
with open(Path(experiment_folder)/'FNOPE_always_equispaced_True/num_sim_100_run_1/predictive_summary.pkl', 'rb') as f1:
    data_FNOfix = pickle.load(f1)

theta_FNOfix = data_FNOfix['theta_test']
x_FNOfix = data_FNOfix['x_test']
posterior_FNOfix = data_FNOfix['posterior_samples']
posterior_predictive_FNOfix = data_FNOfix['posterior_predictive_samples'] # this contains NaNs which need to be filtered out

# Load data for FNOPE
with open(Path(experiment_folder)/'FNOPE_always_equispaced_False/num_sim_100_run_1/predictive_summary.pkl', 'rb') as f2:
    data_FNOflex = pickle.load(f2)

theta_FNOflex = data_FNOflex['theta_test']
x_FNOflex = data_FNOflex['x_test']
posterior_FNOflex = data_FNOflex['posterior_samples']
posterior_predictive_FNOflex = data_FNOflex['posterior_predictive_samples'] # this contains NaNs which need to be filtered out

# Load data for spectral NPE
with open(Path(experiment_folder)/'spectral_NPE/num_sim_100_run_1/predictive_summary.pkl', 'rb') as f3:
    data_specNPE = pickle.load(f3)

theta_specNPE = data_specNPE['theta_test']
x_specNPE = data_specNPE['x_test']
posterior_specNPE = data_specNPE['posterior_samples']
posterior_predictive_specNPE = data_specNPE['posterior_predictive_samples'] # this contains NaNs which need to be filtered out

# Load data for raw FMPE
with open(Path(experiment_folder)/'raw_FMPE/num_sim_100_run_1/predictive_summary.pkl', 'rb') as f4:
    data_rawFMPE = pickle.load(f4)

theta_rawFMPE = data_rawFMPE['theta_test']
x_rawFMPE = data_rawFMPE['x_test']
posterior_rawFMPE = data_rawFMPE['posterior_samples']
posterior_predictive_rawFMPE = data_rawFMPE['posterior_predictive_samples'] # this contains NaNs which need to be filtered out

# Load data for spetral FMPE
with open(Path(experiment_folder)/'spectral_FMPE/num_sim_100_run_1/predictive_summary.pkl', 'rb') as f5:
    data_specFMPE = pickle.load(f5)


theta_specFMPE = data_specFMPE['theta_test']
x_specFMPE = data_specFMPE['x_test']
posterior_specFMPE = data_specFMPE['posterior_samples']
posterior_predictive_specFMPE = data_specFMPE['posterior_predictive_samples'] # this contains NaNs which need to be filtered out


In [ ]:
#### Plot and save observation for overview figure ####

plt.imshow(x_FNOfix[-1,0,:,:], vmin=0, vmax=0.08)
plt.axis('off')
# plt.savefig('darcy_plots/observation.png', dpi=200)

In [ ]:
#### Compute necessary statistics for FNO flex ####

true_theta_flex = np.exp(theta_FNOflex[0,0,:,:])
mean_theta_flex = np.mean(np.exp(posterior_FNOflex[:,0,:,:]), axis=0) # theta should be visualized in physical space
sd_theta_flex = np.std(np.exp(posterior_FNOflex[:,0,:,:]), axis=0)
absolute_error_theta_flex = np.abs(mean_theta_flex - true_theta_flex)
true_observation_flex = x_FNOflex[0,0,:,:]


# Get mean observation (kick out data with NaNs)
posterior_predictive_samples_filtered = np.zeros_like(posterior_predictive_FNOflex[:,0,:,:])

n_skipped_observations = 0 
n_used_layer = 0
for ii in range(posterior_predictive_FNOflex.shape[0]):
    if np.sum(~np.isnan(posterior_predictive_FNOflex[ii,0,:,:])) == 0:
        n_skipped_observations += 1
    elif np.any(posterior_predictive_FNOflex[ii,0,:,:] > 1):
        n_skipped_observations += 1
    else:
        posterior_predictive_samples_filtered[n_used_layer,:,:] = posterior_predictive_FNOflex[ii, 0, :, :]
        n_used_layer += 1 


posterior_predictive_samples_filtered = posterior_predictive_samples_filtered[:n_used_layer,:,:]

mean_observation = np.mean(posterior_predictive_samples_filtered, axis=0)
rand_idx = np.random.randint(0, posterior_predictive_samples_filtered.shape[0])
single_observation = posterior_predictive_samples_filtered[rand_idx, :, :]
relative_error_observation = np.abs((mean_observation - true_observation_flex)/true_observation_flex) 



In [ ]:
#### Compute necessary statistics for FNO fix ####

true_theta_fix = np.exp(theta_FNOfix[0,0,:,:])
mean_theta_fix = np.mean(np.exp(posterior_FNOfix[:,0,:,:]), axis=0) # theta should be visualized in physical space
sd_theta_fix = np.std(np.exp(posterior_FNOfix[:,0,:,:]), axis=0)
absolute_error_theta_fix = np.abs(mean_theta_fix - true_theta_fix)
true_observation_fix = x_FNOfix[0,0,:,:]

In [ ]:
#### Plot of mean theta with crosssections for overview figure ####

plt.imshow(true_theta_fix, vmin=0, vmax=1.8)
plt.axis('off')
# plt.savefig('darcy_plots/true_theta.png', dpi=200)
plt.show()

plt.imshow(np.exp(posterior_FNOfix[10,0,:,:]), vmin=0, vmax=1.8)
plt.axis('off')
plt.axhline(y=40, color='lightgray', linestyle='--', linewidth=5)
plt.axvline(x=30, color='lightgray', linestyle='--', linewidth=5)
# plt.savefig('darcy_plots/theta_posterior_sample.png', dpi=200)
plt.show()

with plt.rc_context(fname='matplotlibrc'):

    plt.plot(np.arange(0,129), mean_theta_fix[40,:], color='black', linewidth=4)
    plt.plot(np.arange(129), mean_theta_fix[40,:] - sd_theta_fix[40,:], color='black', linewidth=0.6)
    plt.plot(np.arange(129), mean_theta_fix[40,:] + sd_theta_fix[40,:], color='black', linewidth=0.6)
    plt.fill_between(np.arange(0,129), mean_theta_fix[40,:] - sd_theta_fix[40,:], mean_theta_fix[40,:] + sd_theta_fix[40,:], color='lightgrey', alpha=1)
    #plt.plot(np.arange(129), true_theta_fix[40,:])
    plt.xlim(0,129)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.set_aspect(50)
    # plt.savefig('darcy_plots/horizontal_uncertainty.png', dpi=200)
    plt.show()


    plt.plot(np.arange(0,129), mean_theta_fix[:,30], color='black', linewidth=4)
    plt.plot(np.arange(129), mean_theta_fix[:,30] - sd_theta_fix[:,30], color='black', linewidth=0.6)
    plt.plot(np.arange(129), mean_theta_fix[:,30] + sd_theta_fix[:,30], color='black', linewidth=0.6)
    plt.fill_between(np.arange(0,129), mean_theta_fix[:,30] - sd_theta_fix[:,30], mean_theta_fix[:,30] + sd_theta_fix[:,30], color='lightgrey', alpha=1)
    plt.xlim(0,129)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.set_aspect(50)

    # plt.savefig('darcy_plots/vertical_uncertainty.png', dpi=200)
    plt.show()




In [ ]:
#### Get MSE and SBC for Darcy ####

path = Path(experiment_folder) / 'summary.csv'
data = pd.read_csv(path)
# data = pd.read_pickle(path)

# Get the different methods 
methods = data['method'].unique()
n_sim = data['nsim'].unique()

# Calculate mean and SE for each method and each number of simulations
mses_results = {}
sbc_results = {}
logprob_results = {}
mses_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
mses_SE = np.zeros((methods.shape[0], n_sim.shape[0]))
sbcs_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
sbcs_SE = np.zeros((methods.shape[0], n_sim.shape[0]))
logprob_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
logprob_SE = np.zeros((methods.shape[0], n_sim.shape[0]))

for ii, method in enumerate(methods):
    print("method: ", method)

    mses_results[method] = {}
    sbc_results[method] = {}
    logprob_results[method] = {}

    for kk, nsim in enumerate(n_sim):

        mses_results[method][nsim] = []
        sbc_results[method][nsim] = []
        logprob_results[method][nsim] = []

        temp_mses = data[(data['method'] == method) & (data['nsim'] == nsim)]['predictive_mses']
        temp_sbcs = data[(data['method'] == method) & (data['nsim'] == nsim)]['sbcs']
        temp_logprob = data[(data['method'] == method) & (data['nsim'] == nsim)]['posterior_log_probs']
        
        for ll in range(temp_mses.shape[0]):
            s_mses = temp_mses.iloc[ll]
            s_mses_clean = s_mses.replace('[', '').replace(']', '')
            list_mses = [float(x) for x in s_mses_clean.split() if x != 'nan']
            filtered_list_mses = [x for x in list_mses if x <= 1] # filter out unreasonable high numbers for MSE
            mses_results[method][nsim].extend(filtered_list_mses)

        mses_mean[ii, kk] = np.mean(np.array(mses_results[method][nsim]))
        mses_SE[ii, kk] = np.std(np.array(mses_results[method][nsim]))/np.sqrt(len(mses_results[method][nsim]))

        for mm in range(temp_sbcs.shape[0]):
            s_sbcs = temp_sbcs.iloc[mm]
            s_sbcs_clean = s_sbcs.replace('[', '').replace(']', '')

            list_sbcs = [float(x) for x in s_sbcs_clean.split() if x != 'nan']
            sbc_results[method][nsim].extend(list_sbcs)

        sbcs_mean[ii, kk] = np.mean(np.array(sbc_results[method][nsim]))
        sbcs_SE[ii, kk] = np.std(np.array(sbc_results[method][nsim]))/np.sqrt(len(sbc_results[method][nsim]))

        if method in ["spectral_NPE", "spectral_FMPE"]:
            continue
            
        else:
            for nn in range(temp_logprob.shape[0]):
                s_logprob = temp_logprob.iloc[mm]
                s_logprob_clean = s_logprob.replace('[', '').replace(']', '')
                list_logprob = [float(x) for x in s_logprob_clean.split() if x != 'nan']
                logprob_results[method][nsim].extend(list_logprob)

            logprob_mean[ii, kk] = np.mean(np.array(logprob_results[method][nsim]))
            logprob_SE[ii, kk] = np.std(np.array(logprob_results[method][nsim]))/np.sqrt(len(logprob_results[method][nsim]))

data

In [ ]:
#### Calculate MSE for inferred thetas to ground truth ####

# Compute the posterior mean for each method and each theta
mean_posterior_FNOfix = np.mean(np.exp(posterior_FNOfix), axis=0)
posterior_MSE_FNOfix = np.sum((mean_posterior_FNOfix - theta_FNOfix[:,0,:,:])**2, axis=(1,2))/posterior_FNOfix.shape[-1]**2
posterior_MSE_FNOfix = np.mean(posterior_MSE_FNOfix)
print(posterior_MSE_FNOfix)

mean_posterior_FNOflex = np.mean(np.exp(posterior_FNOflex), axis=0)
posterior_MSE_FNOflex = np.sum((mean_posterior_FNOflex - theta_FNOflex[:,0,:,:])**2, axis=(1,2))/posterior_FNOflex.shape[-1]**2
posterior_MSE_FNOflex = np.mean(posterior_MSE_FNOflex)
print(posterior_MSE_FNOflex)


mean_posterior_specNPE = np.mean(np.exp(posterior_specNPE), axis=0)
posterior_MSE_specNPE = np.sum((mean_posterior_specNPE - theta_FNOflex[:,0,:,:])**2, axis=(1,2))/posterior_specNPE.shape[-1]**2
posterior_MSE_specNPE = np.mean(posterior_MSE_specNPE)
print(posterior_MSE_specNPE)

mean_posterior_specFMPE = np.mean(np.exp(posterior_specFMPE), axis=0)
posterior_MSE_specFMPE = np.sum((mean_posterior_specFMPE - theta_FNOflex[:,0,:,:])**2, axis=(1,2))/posterior_specFMPE.shape[-1]**2
posterior_MSE_specFMPE = np.mean(posterior_MSE_specFMPE)
print(posterior_MSE_specFMPE)

mean_posterior_rawFMPE = np.mean(np.exp(posterior_rawFMPE), axis=0)
posterior_MSE_rawFMPE = np.sum((mean_posterior_rawFMPE - theta_FNOflex[:,0,:,:])**2, axis=(1,2))/posterior_rawFMPE.shape[-1]**2
posterior_MSE_rawFMPE = np.mean(posterior_MSE_rawFMPE)
print(posterior_MSE_rawFMPE)





In [ ]:
#### Figure for appendix comparing posterior samples between the different methods ####

with plt.rc_context(fname='matplotlibrc'):
    
    plt.tight_layout()
    fig, axs = plt.subplots(6, 6, figsize=(8, 8))
    fig.subplots_adjust(wspace=0.01)
    fig.subplots_adjust(hspace=0.1)

    # Plot ground truth theta
    im1 = axs[0,0].imshow(np.exp(theta_FNOflex[6,0,:,:]), vmin = 0.2, vmax = 1.8,)
    im2 = axs[0,1].imshow(np.exp(theta_FNOflex[7,0,:,:]), vmin = 0.2, vmax = 1.8)
    im3 = axs[0,2].imshow(np.exp(theta_FNOflex[2,0,:,:]), vmin = 0.2, vmax = 1.8)
    im4 = axs[0,3].imshow(np.exp(theta_FNOflex[3,0,:,:]), vmin = 0.2, vmax = 1.8)
    im5 = axs[0,4].imshow(np.exp(theta_FNOflex[4,0,:,:]), vmin = 0.2, vmax = 1.8)
    im6 = axs[0,5].imshow(np.exp(theta_FNOflex[5,0,:,:]), vmin = 0.2, vmax = 1.8)

    axs[0,0].set_ylabel(f'Ground truth', fontsize=9)
    axs[0,0].xaxis.set_label_position('top')  # Move label to top
    axs[0,0].set_xlabel(r'$\theta_1$', labelpad=10, fontsize=10)

    # Plot theta from FNOPE (fix)
    im7 = axs[1,0].imshow(np.exp(posterior_FNOfix[5,6,:,:]), vmin = 0.2, vmax = 1.8)
    im8 = axs[1,1].imshow(np.exp(posterior_FNOfix[2,7,:,:]), vmin = 0.2, vmax = 1.8)
    im9 = axs[1,2].imshow(np.exp(posterior_FNOfix[2,2,:,:]), vmin = 0.2, vmax = 1.8)
    im10 = axs[1,3].imshow(np.exp(posterior_FNOfix[0,3,:,:]), vmin = 0.2, vmax = 1.8)
    im11= axs[1,4].imshow(np.exp(posterior_FNOfix[0,4,:,:]), vmin = 0.2, vmax = 1.8)
    im12 = axs[1,5].imshow(np.exp(posterior_FNOfix[1,5,:,:]), vmin = 0.2, vmax = 1.8)

    axs[1,0].set_ylabel(f'FNOPE (fix)', fontsize=9)
    axs[0,1].xaxis.set_label_position('top')  # Move label to top
    axs[0,1].set_xlabel(r'$\theta_2$', labelpad=10, fontsize=10)

    # Plot theta from FNOPE
    im13 = axs[2,0].imshow(np.exp(posterior_FNOflex[1,6,:,:]), vmin = 0.2, vmax = 1.8)
    im14 = axs[2,1].imshow(np.exp(posterior_FNOflex[6,7,:,:]), vmin = 0.2, vmax = 1.8)
    im15 = axs[2,2].imshow(np.exp(posterior_FNOflex[3,2,:,:]), vmin = 0.2, vmax = 1.8)
    im16 = axs[2,3].imshow(np.exp(posterior_FNOflex[1,3,:,:]), vmin = 0.2, vmax = 1.8)
    im17= axs[2,4].imshow(np.exp(posterior_FNOflex[4,4,:,:]), vmin = 0.2, vmax = 1.8)
    im18 = axs[2,5].imshow(np.exp(posterior_FNOflex[3,5,:,:]), vmin = 0.2, vmax = 1.8)

    axs[2,0].set_ylabel(f'FNOPE', fontsize=9)
    axs[0,2].xaxis.set_label_position('top')  # Move label to top
    axs[0,2].set_xlabel(r'$\theta_3$', labelpad=10, fontsize=10)

    # Plot theta from spectral NPE
    im19 = axs[3,0].imshow(np.exp(posterior_specNPE[0,6,:,:]), vmin = 0.2, vmax = 1.8)
    im20 = axs[3,1].imshow(np.exp(posterior_specNPE[0,7,:,:]), vmin = 0.2, vmax = 1.8)
    im21 = axs[3,2].imshow(np.exp(posterior_specNPE[0,2,:,:]), vmin = 0.2, vmax = 1.8)
    im22 = axs[3,3].imshow(np.exp(posterior_specNPE[0,3,:,:]), vmin = 0.2, vmax = 1.8)
    im23= axs[3,4].imshow(np.exp(posterior_specNPE[0,4,:,:]), vmin = 0.2, vmax = 1.8)
    im24 = axs[3,5].imshow(np.exp(posterior_specNPE[0,5,:,:]), vmin = 0.2, vmax = 1.8)

    axs[3,0].set_ylabel(f'NPE (spectral)', fontsize=9)
    axs[0,3].xaxis.set_label_position('top')  # Move label to top
    axs[0,3].set_xlabel(r'$\theta_5$', labelpad=10, fontsize=10)

    # Plot theta from spectral FMPE
    im25 = axs[4,0].imshow(np.exp(posterior_specFMPE[0,6,:,:]), vmin = 0.2, vmax = 1.8)
    im26 = axs[4,1].imshow(np.exp(posterior_specFMPE[0,7,:,:]), vmin = 0.2, vmax = 1.8)
    im27 = axs[4,2].imshow(np.exp(posterior_specFMPE[0,2,:,:]), vmin = 0.2, vmax = 1.8)
    im28 = axs[4,3].imshow(np.exp(posterior_specFMPE[0,3,:,:]), vmin = 0.2, vmax = 1.8)
    im29= axs[4,4].imshow(np.exp(posterior_specFMPE[0,4,:,:]), vmin = 0.2, vmax = 1.8)
    im30 = axs[4,5].imshow(np.exp(posterior_specFMPE[0,5,:,:]), vmin = 0.2, vmax = 1.8)

    axs[4,0].set_ylabel(f'FMPE (spectral)', fontsize=9)
    axs[0,4].xaxis.set_label_position('top')  # Move label to top
    axs[0,4].set_xlabel(r'$\theta_6$', labelpad=10, fontsize=10)

    # Plot theta from raw FMPE
    im31 = axs[5,0].imshow(np.exp(posterior_rawFMPE[0,6,:,:]), vmin = 0.2, vmax = 1.8)
    im32 = axs[5,1].imshow(np.exp(posterior_rawFMPE[0,7,:,:]), vmin = 0.2, vmax = 1.8)
    im33 = axs[5,2].imshow(np.exp(posterior_rawFMPE[0,2,:,:]), vmin = 0.2, vmax = 1.8)
    im34 = axs[5,3].imshow(np.exp(posterior_rawFMPE[0,3,:,:]), vmin = 0.2, vmax = 1.8)
    im35= axs[5,4].imshow(np.exp(posterior_rawFMPE[0,4,:,:]), vmin = 0.2, vmax = 1.8)
    im36 = axs[5,5].imshow(np.exp(posterior_rawFMPE[0,5,:,:]), vmin = 0.2, vmax = 1.8)

    axs[5,0].set_ylabel(f'FMPE (raw)', fontsize=9)

    for ax in axs.flat:
        #ax.axis('off')
        # Hide spines
        for spine in ax.spines.values():
            spine.set_visible(False)
    
        # Hide ticks and tick labels
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    # Save figure 
    # plt.savefig('darcy_plots/posterior_samples_Darcy.pdf', format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
#### Plot mean posteriors for appendix ####

with plt.rc_context(fname='matplotlibrc'):
    
    plt.tight_layout()
    fig, axs = plt.subplots(6, 6, figsize=(8, 8))
    fig.subplots_adjust(wspace=0.01)
    fig.subplots_adjust(hspace=0.1)

    # Plot ground truth theta
    im1 = axs[0,0].imshow(np.exp(theta_FNOflex[6,0,:,:]), vmin = 0.2, vmax = 1.8,)
    im2 = axs[0,1].imshow(np.exp(theta_FNOflex[7,0,:,:]), vmin = 0.2, vmax = 1.8)
    im3 = axs[0,2].imshow(np.exp(theta_FNOflex[2,0,:,:]), vmin = 0.2, vmax = 1.8)
    im4 = axs[0,3].imshow(np.exp(theta_FNOflex[3,0,:,:]), vmin = 0.2, vmax = 1.8)
    im5 = axs[0,4].imshow(np.exp(theta_FNOflex[4,0,:,:]), vmin = 0.2, vmax = 1.8)
    im6 = axs[0,5].imshow(np.exp(theta_FNOflex[5,0,:,:]), vmin = 0.2, vmax = 1.8)

    axs[0,0].set_ylabel(f'Ground truth', fontsize=9)
    axs[0,0].xaxis.set_label_position('top')  # Move label to top
    axs[0,0].set_xlabel(r'$\theta_1$', labelpad=10, fontsize=10)

    # Plot theta from FNOPE (fix)
    im7 = axs[1,0].imshow(np.mean(np.exp(posterior_FNOfix[:,6,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im8 = axs[1,1].imshow(np.mean(np.exp(posterior_FNOfix[:,7,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im9 = axs[1,2].imshow(np.mean(np.exp(posterior_FNOfix[:,2,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im10 = axs[1,3].imshow(np.mean(np.exp(posterior_FNOfix[:,3,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im11= axs[1,4].imshow(np.mean(np.exp(posterior_FNOfix[:,4,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im12 = axs[1,5].imshow(np.mean(np.exp(posterior_FNOfix[:,5,:,:]), axis=0), vmin = 0.2, vmax = 1.8)

    axs[1,0].set_ylabel(f'FNOPE (fix)', fontsize=9)
    axs[0,1].xaxis.set_label_position('top')  # Move label to top
    axs[0,1].set_xlabel(r'$\theta_2$', labelpad=10, fontsize=10)

    # Plot theta from FNOPE
    im13 = axs[2,0].imshow(np.mean(np.exp(posterior_FNOflex[:,6,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im14 = axs[2,1].imshow(np.mean(np.exp(posterior_FNOflex[:,7,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im15 = axs[2,2].imshow(np.mean(np.exp(posterior_FNOflex[:,2,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im16 = axs[2,3].imshow(np.mean(np.exp(posterior_FNOflex[:,3,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im17= axs[2,4].imshow(np.mean(np.exp(posterior_FNOflex[:,4,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im18 = axs[2,5].imshow(np.mean(np.exp(posterior_FNOflex[:,5,:,:]), axis=0), vmin = 0.2, vmax = 1.8)

    axs[2,0].set_ylabel(f'FNOPE', fontsize=9)
    axs[0,2].xaxis.set_label_position('top')  # Move label to top
    axs[0,2].set_xlabel(r'$\theta_3$', labelpad=10, fontsize=10)

    # Plot theta from spectral NPE
    im25 = axs[3,0].imshow(np.mean(np.exp(posterior_specNPE[:,6,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im26 = axs[3,1].imshow(np.mean(np.exp(posterior_specNPE[:,7,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im27 = axs[3,2].imshow(np.mean(np.exp(posterior_specNPE[:,2,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im28 = axs[3,3].imshow(np.mean(np.exp(posterior_specNPE[:,3,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im29 = axs[3,4].imshow(np.mean(np.exp(posterior_specNPE[:,4,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im30 = axs[3,5].imshow(np.mean(np.exp(posterior_specNPE[:,5,:,:]), axis=0), vmin = 0.2, vmax = 1.8)

    axs[3,0].set_ylabel(f'NPE (spectral)', fontsize=9)
    axs[0,3].xaxis.set_label_position('top')  # Move label to top
    axs[0,3].set_xlabel(r'$\theta_5$', labelpad=10, fontsize=10)

    # Plot theta from spectral FMPE
    im31 = axs[4,0].imshow(np.mean(np.exp(posterior_specFMPE[:,6,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im32 = axs[4,1].imshow(np.mean(np.exp(posterior_specFMPE[:,7,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im33 = axs[4,2].imshow(np.mean(np.exp(posterior_specFMPE[:,2,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im34 = axs[4,3].imshow(np.mean(np.exp(posterior_specFMPE[:,3,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im35 = axs[4,4].imshow(np.mean(np.exp(posterior_specFMPE[:,4,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im36 = axs[4,5].imshow(np.mean(np.exp(posterior_specFMPE[:,5,:,:]), axis=0), vmin = 0.2, vmax = 1.8)

    axs[4,0].set_ylabel(f'FMPE (spectral)', fontsize=9)
    axs[0,4].xaxis.set_label_position('top')  # Move label to top
    axs[0,4].set_xlabel(r'$\theta_6$', labelpad=10, fontsize=10)

    # Plot theta from raw FMPE
    im37 = axs[5,0].imshow(np.mean(np.exp(posterior_rawFMPE[:,6,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im38 = axs[5,1].imshow(np.mean(np.exp(posterior_rawFMPE[:,7,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im39 = axs[5,2].imshow(np.mean(np.exp(posterior_rawFMPE[:,2,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im40 = axs[5,3].imshow(np.mean(np.exp(posterior_rawFMPE[:,3,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im41 = axs[5,4].imshow(np.mean(np.exp(posterior_rawFMPE[:,4,:,:]), axis=0), vmin = 0.2, vmax = 1.8)
    im42 = axs[5,5].imshow(np.mean(np.exp(posterior_rawFMPE[:,5,:,:]), axis=0), vmin = 0.2, vmax = 1.8)

    axs[5,0].set_ylabel(f'FMPE (raw)', fontsize=9)

    for ax in axs.flat:
        #ax.axis('off')
        # Hide spines
        for spine in ax.spines.values():
            spine.set_visible(False)
    
        # Hide ticks and tick labels
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    cbar_mappable = mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=0.2, vmax=1.8), cmap='viridis')

    # Add a colorbar to the right of the entire figure
    cbar_ax = fig.add_axes([0.91, 0.112, 0.015, 0.768])  # [left, bottom, width, height]
    cbar = fig.colorbar(cbar_mappable, cax=cbar_ax, orientation='vertical')
    #cbar.set_label('Permeability', fontsize=9)
    cbar.ax.tick_params(labelsize=9)

    #for spine in cbar.ax.spines.values():
        #spine.set_linewidth(0.5)

    # Save figure 
    # plt.savefig('darcy_plots/posterior_mean_Darcy.pdf', format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
#### Plot sd of posteriors for appendix ####

with plt.rc_context(fname='matplotlibrc'):
    
    plt.tight_layout()
    fig, axs = plt.subplots(5, 6, figsize=(8, 6.66))
    fig.subplots_adjust(wspace=0.01)
    fig.subplots_adjust(hspace=0.1)


    # Plot theta from FNOPE fix
    im7 = axs[0,0].imshow(np.std(np.exp(posterior_FNOfix[:,6,:,:]), axis=0), vmin = 0.0, vmax = 0.5)
    im8 = axs[0,1].imshow(np.std(np.exp(posterior_FNOfix[:,7,:,:]), axis=0), vmin = 0.0, vmax = 0.5)
    im9 = axs[0,2].imshow(np.std(np.exp(posterior_FNOfix[:,2,:,:]), axis=0), vmin = 0.0, vmax = 0.5)
    im10 = axs[0,3].imshow(np.std(np.exp(posterior_FNOfix[:,3,:,:]), axis=0), vmin = 0.0, vmax = 0.5)
    im11= axs[0,4].imshow(np.std(np.exp(posterior_FNOfix[:,4,:,:]), axis=0), vmin = 0.0, vmax = 0.5)
    im12 = axs[0,5].imshow(np.std(np.exp(posterior_FNOfix[:,5,:,:]), axis=0), vmin = 0.0, vmax = 0.5)

    axs[0,0].set_ylabel(f'FNOPE (fix)', fontsize=9)

    axs[0,0].xaxis.set_label_position('top')  # Move label to top
    axs[0,0].set_xlabel(r'$\theta_1$', labelpad=10, fontsize=10)
    axs[0,1].xaxis.set_label_position('top')  # Move label to top
    axs[0,1].set_xlabel(r'$\theta_2$', labelpad=10, fontsize=10)
    axs[0,2].xaxis.set_label_position('top')  # Move label to top
    axs[0,2].set_xlabel(r'$\theta_3$', labelpad=10, fontsize=10)
    axs[0,3].xaxis.set_label_position('top')  # Move label to top
    axs[0,3].set_xlabel(r'$\theta_4$', labelpad=10, fontsize=10)
    axs[0,4].xaxis.set_label_position('top')  # Move label to top
    axs[0,4].set_xlabel(r'$\theta_5$', labelpad=10, fontsize=10)
    axs[0,5].xaxis.set_label_position('top')  # Move label to top
    axs[0,5].set_xlabel(r'$\theta_6$', labelpad=10, fontsize=10)

    # Plot theta from FNOPE flex
    im13 = axs[1,0].imshow(np.std(np.exp(posterior_FNOflex[:,6,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im14 = axs[1,1].imshow(np.std(np.exp(posterior_FNOflex[:,7,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im15 = axs[1,2].imshow(np.std(np.exp(posterior_FNOflex[:,2,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im16 = axs[1,3].imshow(np.std(np.exp(posterior_FNOflex[:,3,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im17= axs[1,4].imshow(np.std(np.exp(posterior_FNOflex[:,4,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im18 = axs[1,5].imshow(np.std(np.exp(posterior_FNOflex[:,5,:,:]), axis=0), vmin = 0.0, vmax = 0.6)

    axs[1,0].set_ylabel(f'FNOPE', fontsize=9)

    # Plot theta from spectral NPE
    im19 = axs[2,0].imshow(np.std(np.exp(posterior_specNPE[:,6,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im20 = axs[2,1].imshow(np.std(np.exp(posterior_specNPE[:,7,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im21 = axs[2,2].imshow(np.std(np.exp(posterior_specNPE[:,2,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im22 = axs[2,3].imshow(np.std(np.exp(posterior_specNPE[:,3,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im23= axs[2,4].imshow(np.std(np.exp(posterior_specNPE[:,4,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im24 = axs[2,5].imshow(np.std(np.exp(posterior_specNPE[:,5,:,:]), axis=0), vmin = 0.0, vmax = 0.6)

    axs[2,0].set_ylabel(f'NPE (spectral)', fontsize=9)

    # Plot theta from spectral FMPE
    im25 = axs[3,0].imshow(np.std(np.exp(posterior_specFMPE[:,6,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im26 = axs[3,1].imshow(np.std(np.exp(posterior_specFMPE[:,7,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im27 = axs[3,2].imshow(np.std(np.exp(posterior_specFMPE[:,2,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im28 = axs[3,3].imshow(np.std(np.exp(posterior_specFMPE[:,3,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im29= axs[3,4].imshow(np.std(np.exp(posterior_specFMPE[:,4,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im30 = axs[3,5].imshow(np.std(np.exp(posterior_specFMPE[:,5,:,:]), axis=0), vmin = 0.0, vmax = 0.6)

    axs[3,0].set_ylabel(f'FMPE (spectral)', fontsize=9)

    # Plot theta from raw FMPE
    im31 = axs[4,0].imshow(np.std(np.exp(posterior_rawFMPE[:,6,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im32 = axs[4,1].imshow(np.std(np.exp(posterior_rawFMPE[:,7,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im33 = axs[4,2].imshow(np.std(np.exp(posterior_rawFMPE[:,2,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im34 = axs[4,3].imshow(np.std(np.exp(posterior_rawFMPE[:,3,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im35= axs[4,4].imshow(np.std(np.exp(posterior_rawFMPE[:,4,:,:]), axis=0), vmin = 0.0, vmax = 0.6)
    im36 = axs[4,5].imshow(np.std(np.exp(posterior_rawFMPE[:,5,:,:]), axis=0), vmin = 0.0, vmax = 0.6)

    axs[4,0].set_ylabel(f'FMPE (raw)', fontsize=9)

    for ax in axs.flat:
        #ax.axis('off')
        # Hide spines
        for spine in ax.spines.values():
            spine.set_visible(False)
    
        # Hide ticks and tick labels
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    cbar_mappable = mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=0.0, vmax=0.6), cmap='viridis')

    # Add a colorbar to the right of the entire figure
    cbar_ax = fig.add_axes([0.91, 0.112, 0.015, 0.768])  # [left, bottom, width, height]
    cbar = fig.colorbar(cbar_mappable, cax=cbar_ax, orientation='vertical')
    #cbar.set_label('Permeability', fontsize=9)
    cbar.ax.tick_params(labelsize=9)

    #for spine in cbar.ax.spines.values():
        #spine.set_linewidth(0.5)

    # Save figure 
    # plt.savefig('darcy_plots/posterior_sd_Darcy.pdf', format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
#### Calculate the minimal achievable SBC from uniform ranks ####

num_posterior_samples = 100
n_sbc = 50
n_sbc_marginals = 50

ranks = np.random.randint(0, num_posterior_samples, size=(n_sbc, n_sbc_marginals))

coverage_values = torch.Tensor(ranks) / num_posterior_samples

atcs = []
absolute_atcs = []

for dim_idx in range(coverage_values.shape[1]):
    # calculate empirical CDF via cumsum and normalize
    hist, alpha_grid = torch.histogram(
    coverage_values[:, dim_idx], density=True, bins=30
    )
    # add 0 to the beginning of the ecp curve to match the alpha grid
    ecp = torch.cat([torch.Tensor([0]), torch.cumsum(hist, dim=0) / hist.sum()])
    atc = (ecp - alpha_grid).mean().item()
    absolute_atc = (ecp - alpha_grid).abs().mean().item()
    atcs.append(atc)
    absolute_atcs.append(absolute_atc)

atcs = torch.tensor(atcs)
absolute_atcs = torch.tensor(absolute_atcs)

mean_absolute_atc = absolute_atcs.mean().numpy()
print(mean_absolute_atc)

In [ ]:
#### Try with svg utils ####

methods_names = ['FNOPE (fix)', 'FNOPE', 'NPE (spectral)', 'FMPE (raw)', 'FMPE (spectral)']
colors1 = ['#CA6702', '#9b2226', '#023e8a', '#00b4d8', '#0077b6', '#d90429']
colors2 = ['#CA6702', '#9b2226', '#023e8a', '#00b4d8', '#0077b6', '#d90429']
colors3 = ['#CA6702', '#9b2226', '#023e8a', '#00b4d8', '#0077b6', '#d90429']


with plt.rc_context(fname="matplotlibrc"):

    plt.tight_layout()

    fig, axs = plt.subplots(1, 3, figsize=(5.5206, 0.9))

    fig.subplots_adjust(wspace=0.7)

    zorder = np.array([5, 4, 3, 2, 1])

    for mm in range(methods.shape[0]):

        axs[0].errorbar(
            n_sim,
            mses_mean[mm, :],
            yerr=mses_SE[mm, :],
            fmt="o",
            linestyle="-",
            color=colors1.pop(0),
            label=methods_names[mm],
            zorder = zorder[mm]
        )
        axs[1].errorbar(
            n_sim,
            sbcs_mean[mm, :],
            yerr=sbcs_SE[mm, :],
            fmt="o",
            linestyle="-",
            color=colors2.pop(0),
            label=methods_names[mm],
            zorder = zorder[mm]
        )
        
        if mm == 2 or mm == 4:
            continue
        else:
            axs[2].errorbar(
                n_sim,
                logprob_mean[mm, :],
                yerr=logprob_SE[mm, :],
                fmt="o",
                linestyle="-",
                color=colors3[mm],
                label=methods_names[mm],
                zorder = zorder[mm]
                )

    axs[0].set_xscale("log")
    axs[0].set_xlabel("# simulations")
    axs[0].set_ylabel("MSE")
    axs[0].set_xticks(n_sim)
    axs[0].set_xlim([890, 11220])
    axs[0].set_yticks([1e-4, 3e-4])
    axs[0].set_ylim([0.9e-4, 4e-4])
    formatter = mticker.ScalarFormatter(useMathText=True)
    formatter.set_scientific(True)
    formatter.set_powerlimits((0, 0))
    axs[0].yaxis.set_major_formatter(formatter)
    axs[0].minorticks_off()
    axs[0].spines["left"].set_position(("outward", 5))  # Move y-axis slightly left
    axs[0].spines["bottom"].set_position(("outward", 5))

    axs[1].hlines(mean_absolute_atc, 1e3, 1e4, linestyle=':', color='black', label='lower bound', linewidth=2.5)
    axs[1].set_xscale("log")
    axs[1].set_xlabel("# simulations")
    axs[1].set_xlim([890, 11220])
    axs[1].set_ylabel("SBC EoD")
    axs[1].set_xticks(n_sim)
    axs[1].set_ylim([0.0, 0.2])
    axs[1].set_yticks([0, 0.2])
    axs[1].minorticks_off()
    axs[1].spines["left"].set_position(("outward", 5))  # Move y-axis slightly left
    axs[1].spines["bottom"].set_position(("outward", 5))

    axs[2].set_xscale("log")
    axs[2].set_xlabel("# simulations")
    axs[2].set_xlim([890, 11220])
    axs[2].set_ylabel("Posterior log prob")
    axs[2].set_xticks(n_sim)
    axs[2].set_yticks([-4, 2])
    axs[2].set_ylim([-4.5, 3])
    #formatter = mticker.ScalarFormatter(useMathText=True)
    #formatter.set_scientific(True)
    #formatter.set_powerlimits((0, 0))
    #axs[2].yaxis.set_major_formatter(formatter)
    axs[2].minorticks_off()
    axs[2].spines["left"].set_position(("outward", 5))  # Move y-axis slightly left
    axs[2].spines["bottom"].set_position(("outward", 5))

    # Add legends
    # Get handles and labels
    handles, labels = axs[1].get_legend_handles_labels()

    custom_order = [1, 2, 3, 5, 4]  # New order of methods_names

    # Reorder handles and labels based on custom order
    handles_global = [handles[i] for i in custom_order]
    labels_global = [labels[i] for i in custom_order]

    handle_ideal = [handles[0]]
    label_ideal = [labels[0]]


    axs[1].legend(handles=handle_ideal,
                  labels=label_ideal,
                  loc='upper right',
                  bbox_to_anchor=(1.1, 1.1),
                  frameon=False,
                  handlelength=1.5
                  )

    

    # Create the legend with the new order
    fig.legend(handles=handles_global,
                  labels=labels_global,
                  loc="upper center",
                  bbox_to_anchor=(0.5, -0.45),
                  ncol=3
                  )


    # plt.savefig('darcy_plots/metricsDarcy.svg', format="svg", bbox_inches="tight")
    plt.show()

In [ ]:
#### Plot posterior samples in different plot ####

with plt.rc_context(fname="matplotlibrc"):

    plt.tight_layout()

    fig, axs = plt.subplots(2, 6, figsize=(5.5206, 1.8))

    fig.subplots_adjust(wspace=0.1, hspace=0.09)

    axs[0,0].imshow(np.exp(theta_FNOflex[0,0,:,:]), vmin = 0.2, vmax = 1.8)
    axs[0,1].imshow(np.exp(posterior_FNOfix[4,0,:,:]), vmin = 0.2, vmax = 1.8)
    axs[0,2].imshow(np.exp(posterior_FNOflex[13,0,:,:]), vmin = 0.2, vmax = 1.8)
    axs[0,3].imshow(np.exp(posterior_specNPE[3,0,:,:]), vmin = 0.2, vmax = 1.8)
    axs[0,4].imshow(np.exp(posterior_specFMPE[3,0,:,:]), vmin = 0.2, vmax = 1.8)
    axs[0,5].imshow(np.exp(posterior_rawFMPE[3,0,:,:]), vmin = 0.2, vmax = 1.8)

    axs[1,0].imshow(np.exp(theta_FNOflex[1,0,:,:]), vmin = 0.2, vmax = 1.8)
    axs[1,1].imshow(np.exp(posterior_FNOfix[0,1,:,:]), vmin = 0.2, vmax = 1.8)
    axs[1,2].imshow(np.exp(posterior_FNOflex[0,1,:,:]), vmin = 0.2, vmax = 1.8)
    axs[1,3].imshow(np.exp(posterior_specNPE[0,1,:,:]), vmin = 0.2, vmax = 1.8)
    axs[1,4].imshow(np.exp(posterior_specFMPE[0,1,:,:]), vmin = 0.2, vmax = 1.8)
    axs[1,5].imshow(np.exp(posterior_rawFMPE[0,1,:,:]), vmin = 0.2, vmax = 1.8)

    axs[0,0].set_ylabel(r'$\theta_1$', fontsize=8)
    axs[0,0].xaxis.set_label_position('top')  
    axs[0,0].set_xlabel('Ground truth', fontsize=7)
    axs[0,1].xaxis.set_label_position('top')  
    axs[0,1].set_xlabel('FNOPE (fix)', fontsize=7)
    axs[0,2].xaxis.set_label_position('top') 
    axs[0,2].set_xlabel('FNOPE', fontsize=7)
    # axs[0,3].xaxis.set_label_position('top')  
    axs[0,3].xaxis.set_label_position('top')  
    axs[0,3].set_xlabel('NPE (spectral)', fontsize=7)
    axs[0,4].xaxis.set_label_position('top')  
    axs[0,4].set_xlabel('FMPE (spectral)', fontsize=7)
    axs[0,5].xaxis.set_label_position('top')  
    axs[0,5].set_xlabel('FMPE (raw)', fontsize=7)

    axs[1,0].set_ylabel(r'$\theta_2$', fontsize=8)
    axs[1,0].xaxis.set_label_position('top')  

    for ax in axs.flat:
        #ax.axis('off')
        ax.set_aspect('equal', adjustable='box')
        # Hide spines
        for spine in ax.spines.values():
            spine.set_visible(False)
        
        # Hide ticks and tick labels
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    for spine in axs[0, 1].spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('#CA6702')  
        spine.set_linewidth(2)      

    for spine in axs[1, 1].spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('#CA6702')  
        spine.set_linewidth(2)  

    for spine in axs[0, 2].spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('#9b2226')  
        spine.set_linewidth(2)   

    for spine in axs[1, 2].spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('#9b2226')  
        spine.set_linewidth(2)   

    # plt.savefig('darcy_plots/samplesDarcy_colored.svg', format="svg", bbox_inches="tight")
    plt.show()


In [ ]:
#### Patch svgs together ####

#base_path = "ice_plots/"
kwargs_text = {"size": "7pt", "font": "Arial", "weight": "800"}


# create new SVG figure
fig = sg.SVGFigure("14cm", "5cm")

# load matpotlib-generated figures
fig0 = sg.fromfile('darcy_plots/rebuttal_samplesDarcy_colored.svg')
fig1 = sg.fromfile('darcy_plots/rebuttal_metricsDarcy.svg')


# get the plot objects
plot0 = fig0.getroot()
plot1 = fig1.getroot()

# get sizes
size0 = get_size_tuple(fig0)
size1 = get_size_tuple(fig1)

# a: posterior samples
plot0.moveto(20, 15)

# b: metrics
plot1.moveto(10, size0[1] + 20)

# add text labels
txt0 = sg.TextElement(8, 25, "a", **kwargs_text)
txt1 = sg.TextElement(8, size0[1] + 25, "b", **kwargs_text)
txt2 = sg.TextElement(120, size0[1] + 25, "c", **kwargs_text)
txt3 = sg.TextElement(241, size0[1] + 25, "d", **kwargs_text)


# append plots and labels to figure
fig.append(
    [
        plot0,
        plot1
    ]
)
fig.append([txt0, txt1, txt2, txt3])

# fig.save('darcy_plots/darcy_joint_colored.svg')
